<a href="https://colab.research.google.com/github/BoonthichaSaejia/start-with-asr/blob/main/AIRAK_week3_OneVsRest_SVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Setup

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import torch
from scipy.io import wavfile
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1 Create Dataframe

In [3]:
vec_list = []
for i in range(10270,10280):
  vec_list.append('id'+str(i))

data = {
	'speaker_id': [],
	'path': []
  }
df = pd.DataFrame(data)



for id in vec_list:
  for i, folder in enumerate(os.listdir(os.path.join('/content/drive/MyDrive/2021_AIRAK/vec_5_sec',id))):
    for j, pt in enumerate(os.listdir(os.path.join('/content/drive/MyDrive/2021_AIRAK/vec_5_sec',id,folder))):
      wav_path= os.path.join('/content/drive/MyDrive/2021_AIRAK/vec_5_sec',id,folder,pt)
      new_row = {'speaker_id':id, 'path':wav_path}
      df      = df.append(new_row, ignore_index=True)

tensor_size=[]
for i in range(len(df)):
  tensor = torch.load(df.at[i,'path'])
  tensor_size.append(tensor.size()[2])
df['tensor_size']=tensor_size

In [4]:
%load_ext google.colab.data_table
df

,speaker_id,path,tensor_size
0,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
1,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,438
2,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
3,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
4,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,418
...,...,...,...
1146,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,414
1147,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,410
1148,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
1149,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498


## 1.1 Filtered with too short file

In [5]:
df_filtered = df[df.tensor_size>497]
df_filtered.index = pd.Index(np.arange(len(df_filtered)))
df_filtered

,speaker_id,path,tensor_size
0,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
1,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
2,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
3,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
4,id10270,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
...,...,...,...
790,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
791,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
792,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498
793,id10279,/content/drive/MyDrive/2021_AIRAK/vec_5_sec/id...,498


# 2 Split Data

In [6]:
X=[]
Y=[]
Path=[]

for i,path in enumerate(df_filtered['path']):
  tensor = torch.load(path)
  X.append(tensor.detach().numpy().flatten())
  Y.append(df_filtered.at[i,'speaker_id'])
  Path.append(path)
  print(i, path)

0 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00034.pt
1 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00050.pt
2 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00042.pt
3 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00013.pt
4 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00012.pt
5 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00054.pt
6 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00018.pt
7 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00006.pt
8 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00024.pt
9 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00053.pt
10 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00040.pt
11 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00003.pt
12 /content/drive/MyDrive/2021_AIRAK/vec_5_sec/id10270/GWXujl-xAVM/00033.pt
13 /content/drive/MyDr

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=69)

# 3 Train Model

## 3.1 SVC(kernal=SVC)

In [8]:
model_1 = OneVsRestClassifier(SVC()).fit(X_train,y_train)
predict_1 = model_1.predict(X_test)

NameError: ignored

In [9]:
print(classification_report(y_test,predict_1))

              precision    recall  f1-score   support

     id10270       0.51      0.66      0.58        29
     id10271       1.00      0.07      0.13        14
     id10272       0.00      0.00      0.00        11
     id10273       0.49      0.57      0.53        53
     id10274       0.00      0.00      0.00        10
     id10275       0.80      0.50      0.62        16
     id10276       0.43      0.75      0.55        40
     id10277       0.50      0.14      0.22        14
     id10278       0.55      0.83      0.66        36
     id10279       0.00      0.00      0.00        16

    accuracy                           0.50       239
   macro avg       0.43      0.35      0.33       239
weighted avg       0.47      0.50      0.44       239



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3.2 LinearSVC

In [ ]:
model_2 = OneVsRestClassifier(LinearSVC(random_state=69)).fit(X_train,y_train)
predict_2 = model_2.predict(X_test)

In [ ]:
print(classification_report(y_test,predict_2))

In [10]:
y_table = pd.DataFrame({'y_test':y_test,'y_predict_svc_rbf':predict_1})
result=[]
for i in range(len(y_table)):
  result.append(y_table.iloc[i,0]==y_table.iloc[i,1])
y_table['result']=result

In [27]:
y_table.y_test.value_counts()

id10273    53
id10276    40
id10278    36
id10270    29
id10279    16
id10275    16
id10277    14
id10271    14
id10272    11
id10274    10
Name: y_test, dtype: int64

Assumption
* ข้อมูลน้อยเกินไป
* 10273 มีจำนวนเยอะกว่าหลายเท่า

# Still Don't Use


In [17]:
y_df = pd.read_csv('/content/drive/MyDrive/2021_AIRAK/y_linearSVC.csv')

In [18]:
y_df

,y_test,y_predict,result
0,id10277,id10277,True
1,id10272,id10276,False
2,id10273,id10279,False
3,id10279,id10270,False
4,id10270,id10270,True
...,...,...,...
234,id10276,id10278,False
235,id10271,id10276,False
236,id10279,id10276,False
237,id10279,id10279,True


In [19]:
y_table['y_predict_linearsvc']=y_df['y_predict']

In [21]:
y_table = y_table.drop('result',axis=1)

In [22]:
result=[]
for i in range(len(y_table)):
  result.append(y_table.iloc[i,0]==y_table.iloc[i,1]==y_table.iloc[i,2])
y_table['result']=result

In [25]:
y_table.result.value_counts()

False    128
True     111
Name: result, dtype: int64